<a href="https://colab.research.google.com/github/mugalan/working/blob/main/hierarchical_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

* https://arxiv.org/pdf/2303.03293
* https://ris.utwente.nl/ws/files/52018562/Kienreich2012graph.pdf

# Python Class Definition

In [ ]:
import networkx as nx
import graphviz
from IPython.display import display, Image

class HierarchicalGraph:
    def __init__(self, nodes_data, edges_data):
        self.nodes_data = nodes_data
        self.edges_data = edges_data
        self.inner_graph = nx.MultiGraph()
        self.outer_graph = nx.Graph()
        self.create_hierarchical_graphs_iterative()

    def create_hierarchical_graphs_iterative(self):
        inner_graph = nx.MultiGraph()

        for node_data in self.nodes_data:
            label = node_data['label']
            attributes = {k: v for k, v in node_data.items() if k != 'label'}
            inner_graph.add_node(label, **attributes)

        for edge_data in self.edges_data:
            start = edge_data['start']
            end = edge_data['end']
            attributes = {k: v for k, v in edge_data.items() if k not in ['start', 'end']}
            inner_graph.add_edge(start, end, **attributes)

        outer_graph = nx.Graph()
        meta_edges_data = {}

        for u, v, data in inner_graph.edges(data=True):
            group_u = inner_graph.nodes[u]['group']
            group_v = inner_graph.nodes[v]['group']

            if group_u != group_v:
                key = tuple(sorted((group_u, group_v)))
                if key not in meta_edges_data:
                    meta_edges_data[key] = {'count': 0, 'connection_types': set(), 'total_weight': 0.0}

                meta_edges_data[key]['count'] += 1
                meta_edges_data[key]['connection_types'].add(data.get('type', 'unknown'))
                meta_edges_data[key]['total_weight'] += data.get('weight', 1.0)

        outer_graph.add_nodes_from(set(d['group'] for d in self.nodes_data))

        for (u_group, v_group), data in meta_edges_data.items():
            data['connection_types'] = list(data['connection_types'])
            data['average_weight'] = data['total_weight'] / data['count']
            outer_graph.add_edge(u_group, v_group, **data)

        self.inner_graph = inner_graph
        self.outer_graph = outer_graph

    # --------------------------
    # Node Operations
    # --------------------------
    def add_node(self, node_data):
        self.nodes_data.append(node_data)
        self.create_hierarchical_graphs_iterative()

    def edit_node(self, label, new_data):
        for node in self.nodes_data:
            if node['label'] == label:
                node.update(new_data)
                break
        self.create_hierarchical_graphs_iterative()

    def delete_node(self, label):
        self.nodes_data = [node for node in self.nodes_data if node['label'] != label]
        self.edges_data = [edge for edge in self.edges_data if edge['start'] != label and edge['end'] != label]
        self.create_hierarchical_graphs_iterative()

    # --------------------------
    # Edge Operations
    # --------------------------
    def add_edge(self, edge_data):
        self.edges_data.append(edge_data)
        self.create_hierarchical_graphs_iterative()

    def edit_edge(self, start, end, new_data):
        for edge in self.edges_data:
            if edge['start'] == start and edge['end'] == end:
                edge.update(new_data)
                break
        self.create_hierarchical_graphs_iterative()

    def delete_edge(self, start, end):
        self.edges_data = [edge for edge in self.edges_data if not (edge['start'] == start and edge['end'] == end)]
        self.create_hierarchical_graphs_iterative()

    # --------------------------
    # Visualization Methods
    # --------------------------
    def visualize_outer_graph(self, filename='outer_level_graph'):
        dot = graphviz.Graph(comment='Outer Level Graph', engine='dot')

        for node in self.outer_graph.nodes:
            dot.node(str(node))

        for u, v, data in self.outer_graph.edges(data=True):
            label = f"Count: {data['count']}\\nTypes: {', '.join(data['connection_types'])}"
            dot.edge(str(u), str(v), label=label)

        filepath = dot.render(filename, format='png', cleanup=True)
        print(f"Outer-level graph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the graph.")

    def visualize_inner_graph_with_clusters(self, filename="inner_level_graph"):
        dot_source = 'graph G {\n'
        dot_source += '    rankdir="LR";\n'
        dot_source += '    node [shape=box, style="filled", fillcolor="white"];\n'

        clusters = {}
        for node, attrs in self.inner_graph.nodes(data=True):
            group = attrs.get('group', 'DefaultGroup')
            clusters.setdefault(group, []).append(node)

        colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightpink', 'lightgray']
        for i, (group, nodes) in enumerate(clusters.items()):
            color = colors[i % len(colors)]
            cluster_id = group.replace(" ", "_")
            dot_source += f'    subgraph cluster_{cluster_id} {{\n'
            dot_source += f'        label="{group}";\n'
            dot_source += f'        bgcolor="{color}";\n'
            dot_source += '        ' + ' '.join(f'"{node}"' for node in nodes) + ';\n'
            dot_source += '    }\n'

        for u, v, key, data in self.inner_graph.edges(keys=True, data=True):
            label = data.get('type', '')
            penwidth = str(data.get('weight', 1.0) * 2)
            dot_source += f'    "{u}" -- "{v}" [label="{label}", id="{key}", penwidth={penwidth}];\n'

        dot_source += '}\n'

        dot = graphviz.Source(dot_source, format='png')
        filepath = dot.render(filename, cleanup=True)
        print(f"Inner-level graph saved as {filepath}")
        try:
            display(Image(filename=filepath))
        except:
            print("Open the saved image to view the graph.")


In [ ]:
nodes = [
    {'label': 'A', 'group': 'Group A', 'type': 'system'},
    {'label': 'B', 'group': 'Group A', 'type': 'user'},
    {'label': 'C', 'group': 'Group B', 'type': 'system'},
    {'label': 'D', 'group': 'Group B', 'type': 'user'},
    {'label': 'E', 'group': 'Group C', 'type': 'system'},
]

edges = [
    {'start': 'A', 'end': 'B', 'type': 'type1', 'weight': 0.4},
    {'start': 'A', 'end': 'C', 'type': 'type2', 'weight': 0.8},
    {'start': 'B', 'end': 'C', 'type': 'type1', 'weight': 0.5},
    {'start': 'C', 'end': 'E', 'type': 'type3', 'weight': 0.1},
    {'start': 'C', 'end': 'E', 'type': 'type1', 'weight': 0.9}, # Multi-edge here
    {'start': 'D', 'end': 'E', 'type': 'type2', 'weight': 0.3},
]

In [ ]:
hg = HierarchicalGraph(nodes, edges)
hg.visualize_inner_graph_with_clusters()
hg.visualize_outer_graph()

In [ ]:
hg.add_node({'label': 'F', 'group': 'Group A', 'type': 'user'})

In [ ]:
hg.add_edge(edge_data={'start': 'A', 'end': 'F', 'type': 'type1', 'weight': 0.4})

In [ ]:
hg.visualize_inner_graph_with_clusters()